In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
train=pd.read_csv('/kaggle/input/amazon-ml-challenge-2024/student_resource 3/dataset/train.csv')

In [ ]:
train

In [ ]:
test=pd.read_csv("/kaggle/input/amazon-ml-challenge-2024/student_resource 3/dataset/test.csv")

In [ ]:
test

In [ ]:
# More robust version - handle groups with < 4 samples separately
# (need at least ~4 to safely appear in both temp splits)
group_counts = train['group_id'].value_counts()

In [ ]:
group_counts

In [ ]:
rare_groups = group_counts[group_counts < 2].index
common_groups = group_counts[group_counts >= 2].index

In [ ]:
rare_df = train[train['group_id'].isin(rare_groups)]
main_df = train[train['group_id'].isin(common_groups)]

In [ ]:
rare_df

In [ ]:
main_df['group_id'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Stratified split only on common groups
train_df, temp_df = train_test_split(
    main_df, test_size=0.10, stratify=main_df['group_id'], random_state=42
)

In [ ]:
temp_df

In [ ]:
temp_df['group_id'].value_counts()

In [ ]:
group_counts_2 = temp_df['group_id'].value_counts()

In [ ]:
rare_groups_2 = group_counts_2[group_counts_2 < 2].index
common_groups_2 = group_counts_2[group_counts_2 >= 2].index

In [ ]:
rare_df_2 = temp_df[temp_df['group_id'].isin(rare_groups_2)]
main_df_2 = temp_df[temp_df['group_id'].isin(common_groups_2)]

In [ ]:
val_df, test_df = train_test_split(
    main_df_2, test_size=0.50, stratify=main_df_2['group_id'], random_state=42
)

In [ ]:
val_df

In [ ]:
test_df

In [ ]:
train_df

In [ ]:
train_df = pd.concat([train_df, rare_df,rare_df_2], ignore_index=True)

In [ ]:
train_df

In [ ]:
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
# All rare groups go to train
print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")
print(f"Rare groups pushed to train: {len(rare_df)+len(rare_df_2)}")
print(f"Total check: {len(train_df) + len(val_df) + len(test_df) == len(train)}")

In [ ]:
train_df.to_csv("/kaggle/working/train_df.csv",index=False)

In [ ]:
val_df.to_csv("/kaggle/working/val_df.csv",index=False)

In [ ]:
test_df.to_csv("/kaggle/working/test_df.csv",index=False)